In [1]:
import pyomo

In [69]:
import pandas as pd
from pyomo.environ import *

data = pd.read_csv('final_final.csv')
players = data['playerid'].unique()
teams = data['team_id'].unique()
gameweeks = list(range(1, 39))  
model = ConcreteModel()
model.P = Set(initialize=players)  
model.GW = Set(initialize=gameweeks)  
model.z = Var(model.P, model.GW, domain=Binary)  
cost = {row['playerid']: row['value'] for _, row in data.iterrows()}
points = {(row['playerid'], gw): row[f'GW_{gw}'] for _, row in data.iterrows() for gw in gameweeks}
team = {row['playerid']: row['team_id'] for _, row in data.iterrows()}
position = {row['playerid']: row['position_id'] for _, row in data.iterrows()}

model.x = Var(model.P, model.GW, domain=Binary) 
model.captain = Var(model.P, model.GW, domain=Binary) 

def total_points_rule(model):
    return sum(model.x[p, gw] * points[p, gw] + model.captain[p, gw] * points[p, gw] 
               for p in model.P for gw in model.GW)
model.total_points = Objective(rule=total_points_rule, sense=maximize)

def team_size_rule(model, gw):
    return sum(model.x[p, gw] for p in model.P) == 11
model.team_size = Constraint(model.GW, rule=team_size_rule)

def cost_constraint_rule(model, gw):
    return sum(model.x[p, gw] * cost[p] for p in model.P) <= 85
model.cost_constraint = Constraint(model.GW, rule=cost_constraint_rule)

def team_limit_rule(model, team_id, gw):
    return sum(model.x[p, gw] for p in model.P if team[p] == team_id) <= 4
model.team_limit = Constraint(teams, model.GW, rule=team_limit_rule)

def goalkeeper_rule(model, gw):
    return sum(model.x[p, gw] for p in model.P if position[p] == 3) == 1
model.goalkeeper_constraint = Constraint(model.GW, rule=goalkeeper_rule)

def defenders_rule(model, gw):
    return inequality(3, sum(model.x[p, gw] for p in model.P if position[p] == 2), 5)
model.defenders_constraint = Constraint(model.GW, rule=defenders_rule)

def midfielders_rule(model, gw):
    return inequality(2, sum(model.x[p, gw] for p in model.P if position[p] == 4), 5)
model.midfielders_constraint = Constraint(model.GW, rule=midfielders_rule)

def attackers_rule(model, gw):
    return inequality(1, sum(model.x[p, gw] for p in model.P if position[p] == 1), 3)
model.attackers_constraint = Constraint(model.GW, rule=attackers_rule)

def captain_rule(model, gw):
    return sum(model.captain[p, gw] for p in model.P) == 1
model.captain_constraint = Constraint(model.GW, rule=captain_rule)

def captain_selection_rule(model, p, gw):
    return model.captain[p, gw] <= model.x[p, gw]
model.captain_selection_constraint = Constraint(model.P, model.GW, rule=captain_selection_rule)

def transfer_indicator_rule(model, p, gw):
    if gw == 1:
        return Constraint.Skip
    return model.z[p, gw] >= model.x[p, gw] - model.x[p, gw-1]
model.transfer_indicator_1 = Constraint(model.P, model.GW, rule=transfer_indicator_rule)

def transfer_indicator_rule_2(model, p, gw):
    if gw == 1:
        return Constraint.Skip
    return model.z[p, gw] >= model.x[p, gw-1] - model.x[p, gw]
model.transfer_indicator_2 = Constraint(model.P, model.GW, rule=transfer_indicator_rule_2)

def transfer_limit_rule(model, gw):
    if gw == 1:
        return Constraint.Skip
    return sum(model.z[p, gw] for p in model.P) <= 2
model.transfer_limit = Constraint(model.GW, rule=transfer_limit_rule)

# def transfer_rule(model, gw1, gw2):
#     if gw2 != gw1 + 1:
#         return Constraint.Skip
#     return sum(model.x[p, gw1] - model.x[p, gw2] for p in model.P) <= 2
# model.transfer_constraint = Constraint(model.GW, model.GW, rule=transfer_rule)

# def transfer_rule2(model, gw1, gw2):
#     if gw2 != gw1 - 1:
#         return Constraint.Skip
#     return sum(model.x[p, gw1] - model.x[p, gw2] for p in model.P) <= 2
# model.transfer_constraint = Constraint(model.GW, model.GW, rule=transfer_rule2)

solver = SolverFactory('cbc')
solver.solve(model)

selected_players = {(p, gw): model.x[p, gw].value for p in model.P for gw in model.GW if model.x[p, gw].value == 1}
captains = {(p, gw): model.captain[p, gw].value for p in model.P for gw in model.GW if model.captain[p, gw].value == 1}

print("Selected players each gameweek:", selected_players)
print("Captains each gameweek:", captains)


Selected players each gameweek: {(28, 30): 1.0, (28, 31): 1.0, (28, 32): 1.0, (28, 33): 1.0, (28, 34): 1.0, (28, 35): 1.0, (28, 36): 1.0, (28, 37): 1.0, (28, 38): 1.0, (62, 3): 1.0, (62, 4): 1.0, (62, 5): 1.0, (62, 6): 1.0, (62, 7): 1.0, (62, 8): 1.0, (62, 9): 1.0, (62, 10): 1.0, (62, 11): 1.0, (62, 12): 1.0, (62, 13): 1.0, (62, 14): 1.0, (62, 15): 1.0, (98, 24): 1.0, (98, 25): 1.0, (98, 26): 1.0, (98, 27): 1.0, (98, 28): 1.0, (98, 29): 1.0, (98, 30): 1.0, (98, 31): 1.0, (98, 32): 1.0, (98, 33): 1.0, (98, 34): 1.0, (98, 35): 1.0, (98, 36): 1.0, (98, 37): 1.0, (98, 38): 1.0, (101, 1): 1.0, (101, 2): 1.0, (101, 3): 1.0, (101, 4): 1.0, (101, 5): 1.0, (101, 6): 1.0, (117, 31): 1.0, (117, 32): 1.0, (117, 33): 1.0, (117, 34): 1.0, (121, 1): 1.0, (121, 2): 1.0, (121, 3): 1.0, (121, 4): 1.0, (121, 5): 1.0, (121, 6): 1.0, (121, 7): 1.0, (121, 8): 1.0, (121, 9): 1.0, (121, 10): 1.0, (122, 21): 1.0, (122, 22): 1.0, (122, 23): 1.0, (122, 24): 1.0, (122, 25): 1.0, (122, 26): 1.0, (122, 27): 1.0, (1

In [70]:
def print_teams(selected_players, captains, data):
    player_name_map = {row['playerid']: row['name'] for _, row in data.iterrows()}
    player_position_map = {row['playerid']: row['position'] for _, row in data.iterrows()}

    for gw in range(1, 39):  
        print(f"--- Gameweek {gw} ---")
        
        gw_players = [p for p, gwk in selected_players.keys() if gwk == gw]
        
        if not gw_players:
            print("No players selected for this gameweek.")
            continue

        team = {"Goalkeeper": [], "Defender": [], "Midfielder": [], "Attacker": []}
        
        for player in gw_players:
            position = player_position_map[player]
            name = player_name_map[player]
            if position == 'GK':
                team["Goalkeeper"].append(name)
            elif position == 'DEF':
                team["Defender"].append(name)
            elif position == 'MID':
                team["Midfielder"].append(name)
            elif position == 'FWD':
                team["Attacker"].append(name)

        print("Goalkeeper:")
        for gk in team["Goalkeeper"]:
            print(f"  - {gk}")

        print("Defenders:")
        for df in team["Defender"]:
            print(f"  - {df}")

        print("Midfielders:")
        for mf in team["Midfielder"]:
            print(f"  - {mf}")

        print("Attackers:")
        for fw in team["Attacker"]:
            print(f"  - {fw}")

        gw_captain = [p for p, gwk in captains.keys() if gwk == gw]
        if gw_captain:
            captain_name = player_name_map[gw_captain[0]]
            print(f"Captain: {captain_name} (Points doubled)")

        print("\n")

print_teams(selected_players, captains, data)


--- Gameweek 1 ---
Goalkeeper:
  - Bernd Leno
Defenders:
  - Cristian Romero
  - Joachim Andersen
  - Raphaël Varane
Midfielders:
  - Bryan Mbeumo
  - Jarrod Bowen
  - Mohamed Salah
  - Rodrigo Hernandez
  - Solly March
Attackers:
  - Erling Haaland
  - Julián Álvarez
Captain: Raphaël Varane (Points doubled)


--- Gameweek 2 ---
Goalkeeper:
  - Bernd Leno
Defenders:
  - Cristian Romero
  - Joachim Andersen
  - Matty Cash
Midfielders:
  - Bryan Mbeumo
  - Jarrod Bowen
  - Mohamed Salah
  - Rodrigo Hernandez
  - Solly March
Attackers:
  - Erling Haaland
  - Julián Álvarez
Captain: Bryan Mbeumo (Points doubled)


--- Gameweek 3 ---
Goalkeeper:
  - Bernd Leno
Defenders:
  - Cristian Romero
  - Joachim Andersen
  - Matty Cash
Midfielders:
  - Anthony Gordon
  - Bryan Mbeumo
  - Jarrod Bowen
  - Mohamed Salah
  - Rodrigo Hernandez
Attackers:
  - Erling Haaland
  - Julián Álvarez
Captain: Matty Cash (Points doubled)


--- Gameweek 4 ---
Goalkeeper:
  - Bernd Leno
Defenders:
  - Cristian Romer

In [57]:
import pandas as pd
from pyomo.environ import *

data = pd.read_csv('final_final.csv')

players = data['playerid'].unique()
teams = data['team_id'].unique()
gameweeks = list(range(2, 39))  

model = ConcreteModel()

# Sets
model.P = Set(initialize=players)  
model.GW = Set(initialize=gameweeks)  

# Parameters
cost = {row['playerid']: row['value'] for _, row in data.iterrows()}
points = {(row['playerid'], gw): row[f'GW_{gw}'] for _, row in data.iterrows() for gw in range(1, 39)}
team = {row['playerid']: row['team_id'] for _, row in data.iterrows()}
position = {row['playerid']: row['position_id'] for _, row in data.iterrows()}

# Decision Variables
model.x = Var(model.P, model.GW, domain=Binary)  # Selection of player in a gameweek
model.captain = Var(model.P, model.GW, domain=Binary)  # Selection of captain in a gameweek
model.z = Var(model.P, model.GW, domain=Binary)  # Transfer indicator between consecutive gameweeks

# Objective: Maximize total points for each gameweek based on past data
def total_points_rule(model):
    return sum(model.x[p, gw] * sum(points[p, gwk] * gwk for gwk in range(1, gw)) + 
               model.captain[p, gw] * sum(points[p, gwk] * gwk for gwk in range(1, gw)) 
               for p in model.P for gw in model.GW)
model.total_points = Objective(rule=total_points_rule, sense=maximize)


def team_size_rule(model, gw):
    return sum(model.x[p, gw] for p in model.P) == 11
model.team_size = Constraint(model.GW, rule=team_size_rule)

def cost_constraint_rule(model, gw):
    return sum(model.x[p, gw] * cost[p] for p in model.P) <= 85
model.cost_constraint = Constraint(model.GW, rule=cost_constraint_rule)

def team_limit_rule(model, team_id, gw):
    return sum(model.x[p, gw] for p in model.P if team[p] == team_id) <= 4
model.team_limit = Constraint(teams, model.GW, rule=team_limit_rule)

def goalkeeper_rule(model, gw):
    return sum(model.x[p, gw] for p in model.P if position[p] == 3) == 1
model.goalkeeper_constraint = Constraint(model.GW, rule=goalkeeper_rule)

def defenders_rule(model, gw):
    return inequality(3, sum(model.x[p, gw] for p in model.P if position[p] == 2), 5)
model.defenders_constraint = Constraint(model.GW, rule=defenders_rule)

def midfielders_rule(model, gw):
    return inequality(2, sum(model.x[p, gw] for p in model.P if position[p] == 4), 5)
model.midfielders_constraint = Constraint(model.GW, rule=midfielders_rule)

def attackers_rule(model, gw):
    return inequality(1, sum(model.x[p, gw] for p in model.P if position[p] == 1), 3)
model.attackers_constraint = Constraint(model.GW, rule=attackers_rule)

def captain_rule(model, gw):
    return sum(model.captain[p, gw] for p in model.P) == 1
model.captain_constraint = Constraint(model.GW, rule=captain_rule)

def captain_selection_rule(model, p, gw):
    return model.captain[p, gw] <= model.x[p, gw]
model.captain_selection_constraint = Constraint(model.P, model.GW, rule=captain_selection_rule)

def transfer_indicator_rule(model, p, gw):
    if gw == 2:
        return Constraint.Skip
    return model.z[p, gw] >= model.x[p, gw] - model.x[p, gw-1]
model.transfer_indicator_1 = Constraint(model.P, model.GW, rule=transfer_indicator_rule)

def transfer_indicator_rule_2(model, p, gw):
    if gw == 2:
        return Constraint.Skip
    return model.z[p, gw] >= model.x[p, gw-1] - model.x[p, gw]
model.transfer_indicator_2 = Constraint(model.P, model.GW, rule=transfer_indicator_rule_2)

def transfer_limit_rule(model, gw):
    if gw == 2:
        return Constraint.Skip
    return sum(model.z[p, gw] for p in model.P) <= 1
model.transfer_limit = Constraint(model.GW, rule=transfer_limit_rule)

solver = SolverFactory('cbc')
solver.solve(model)

selected_players = {(p, gw): model.x[p, gw].value for p in model.P for gw in model.GW if model.x[p, gw].value == 1}
captains = {(p, gw): model.captain[p, gw].value for p in model.P for gw in model.GW if model.captain[p, gw].value == 1}
number_of_tranfers = {gw: sum(model.z[p, gw].value for p in model.P if model.z[p, gw].value is not None) for gw in model.GW}
print(number_of_tranfers)
print_teams(selected_players, captains, data)


KeyboardInterrupt: 

In [30]:
transfer_weeks = {gw: [] for gw in gameweeks[1:]}  

for gw in gameweeks[1:]:
    for p in players:
        if model.x[p, gw].value != model.x[p, gw - 1].value:
            transfer_weeks[gw].append(p)

print("Transfers occurred in the following weeks:")
for gw, transferred_players in transfer_weeks.items():
    if transferred_players:  # Only print weeks with transfers
        print(f"Gameweek {gw}: Transfers: {', '.join(transferred_players)}")

Transfers occurred in the following weeks:


In [48]:
import pandas as pd
from pyomo.environ import *

# Load the data from CSV
data = pd.read_csv('final_final.csv')

# Extract unique player ids, team ids, and gameweeks
players = data['playerid'].unique()
teams = data['team_id'].unique()

# Initialize model
model = ConcreteModel()

# Sets
model.P = Set(initialize=players)  # Set of players
model.GW = Set(initialize=gameweeks)  # Set of gameweeks
model.z = Var(model.P, model.GW, domain=Binary)  # Transfer indicator between consecutive gameweeks

# Parameters
cost = {row['playerid']: row['value'] for _, row in data.iterrows()}
points = {(row['playerid'], gw): row[f'GW_{gw}'] for _, row in data.iterrows() for gw in gameweeks}
team = {row['playerid']: row['team_id'] for _, row in data.iterrows()}
position = {row['playerid']: row['position_id'] for _, row in data.iterrows()}

# Decision Variables
model.x = Var(model.P, model.GW, domain=Binary)  # Selection of player in a gameweek
model.captain = Var(model.P, model.GW, domain=Binary)  # Selection of captain in a gameweek

# Objective: Maximize total points (with captain's points doubled)
def total_points_rule(model):
    return sum(model.x[p, gw] * points[p, gw] +  model.captain[p, gw] * points[p, gw]
               for p in model.P for gw in model.GW)
model.total_points = Objective(rule=total_points_rule, sense=maximize)

# Constraints

# Team size constraint: Exactly 11 players per gameweek
def team_size_rule(model, gw):
    return sum(model.x[p, gw] for p in model.P) == 11
model.team_size = Constraint(model.GW, rule=team_size_rule)

# Cost constraint: Total cost < 85 for each gameweek
def cost_constraint_rule(model, gw):
    return sum(model.x[p, gw] * cost[p] for p in model.P) <= 85
model.cost_constraint = Constraint(model.GW, rule=cost_constraint_rule)

# Maximum 4 players from any team per gameweek
def team_limit_rule(model, team_id, gw):
    return sum(model.x[p, gw] for p in model.P if team[p] == team_id) <= 4
model.team_limit = Constraint(teams, model.GW, rule=team_limit_rule)

# Exactly 1 goalkeeper per gameweek
def goalkeeper_rule(model, gw):
    return sum(model.x[p, gw] for p in model.P if position[p] == 3) == 1
model.goalkeeper_constraint = Constraint(model.GW, rule=goalkeeper_rule)

# Defenders between 3 and 5 per gameweek
def defenders_rule(model, gw):
    return inequality(3, sum(model.x[p, gw] for p in model.P if position[p] == 2), 5)
model.defenders_constraint = Constraint(model.GW, rule=defenders_rule)

# Midfielders between 2 and 5 per gameweek
def midfielders_rule(model, gw):
    return inequality(2, sum(model.x[p, gw] for p in model.P if position[p] == 4), 5)
model.midfielders_constraint = Constraint(model.GW, rule=midfielders_rule)

# Attackers between 1 and 3 per gameweek
def attackers_rule(model, gw):
    return inequality(1, sum(model.x[p, gw] for p in model.P if position[p] == 1), 3)
model.attackers_constraint = Constraint(model.GW, rule=attackers_rule)

# Exactly 1 captain per gameweek
def captain_rule(model, gw):
    return sum(model.captain[p, gw] for p in model.P) == 1
model.captain_constraint = Constraint(model.GW, rule=captain_rule)

# A player can only be captain if they are selected
def captain_selection_rule(model, p, gw):
    return model.captain[p, gw] <= model.x[p, gw]
model.captain_selection_constraint = Constraint(model.P, model.GW, rule=captain_selection_rule)
# Transfer constraint: Between two gameweeks, at most one player can be transferred
# We will now use binary variable z[p, gw] to represent transfers

# Introduce the transfer variable constraints
# def transfer_indicator_rule(model, p, gw):
#     if gw == 1:
#         return Constraint.Skip
#     return model.z[p, gw] >= model.x[p, gw] - model.x[p, gw-1]
# model.transfer_indicator_1 = Constraint(model.P, model.GW, rule=transfer_indicator_rule)

# def transfer_indicator_rule_2(model, p, gw):
#     if gw == 1:
#         return Constraint.Skip
#     return model.z[p, gw] >= model.x[p, gw-1] - model.x[p, gw]
# model.transfer_indicator_2 = Constraint(model.P, model.GW, rule=transfer_indicator_rule_2)

# # Limit the total number of transfers between two consecutive gameweeks to at most 1
# def transfer_limit_rule(model, gw):
#     if gw == 1:
#         return Constraint.Skip
#     return sum(model.z[p, gw] for p in model.P) <= 1
# model.transfer_limit = Constraint(model.GW, rule=transfer_limit_rule)



In [49]:
gameweeks = list(range(1, 39))  # 38 gameweeks (GW_1 to GW_38)
print(SolverFactory('cbc').available())
solver = SolverFactory('cbc') # Specify path
solver.solve(model)

# Extract solution
selected_players = {(p, gw): model.x[p, gw].value for p in model.P for gw in model.GW if model.x[p, gw].value == 1}
captains = {(p, gw): model.captain[p, gw].value for p in model.P for gw in model.GW if model.captain[p, gw].value == 1}

print("Selected players each gameweek:", selected_players)
print("Captains each gameweek:", captains)


True
Selected players each gameweek: {(6, 1): 1.0, (7, 12): 1.0, (7, 16): 1.0, (11, 31): 1.0, (20, 19): 1.0, (20, 23): 1.0, (20, 31): 1.0, (23, 15): 1.0, (28, 1): 1.0, (28, 8): 1.0, (28, 30): 1.0, (28, 31): 1.0, (28, 33): 1.0, (28, 35): 1.0, (28, 38): 1.0, (29, 17): 1.0, (29, 23): 1.0, (30, 25): 1.0, (32, 18): 1.0, (32, 20): 1.0, (35, 4): 1.0, (35, 9): 1.0, (38, 19): 1.0, (38, 20): 1.0, (38, 27): 1.0, (40, 37): 1.0, (42, 17): 1.0, (49, 15): 1.0, (49, 33): 1.0, (51, 5): 1.0, (51, 36): 1.0, (53, 18): 1.0, (54, 13): 1.0, (54, 17): 1.0, (54, 28): 1.0, (54, 38): 1.0, (57, 19): 1.0, (61, 19): 1.0, (62, 13): 1.0, (62, 33): 1.0, (64, 13): 1.0, (65, 28): 1.0, (66, 29): 1.0, (76, 1): 1.0, (82, 30): 1.0, (90, 14): 1.0, (98, 7): 1.0, (98, 8): 1.0, (98, 27): 1.0, (98, 28): 1.0, (98, 34): 1.0, (100, 5): 1.0, (100, 11): 1.0, (100, 19): 1.0, (101, 1): 1.0, (101, 6): 1.0, (101, 29): 1.0, (101, 33): 1.0, (105, 4): 1.0, (116, 18): 1.0, (117, 3): 1.0, (117, 11): 1.0, (117, 28): 1.0, (117, 33): 1.0, (117, 

In [50]:
print_teams(selected_players, captains, data)


--- Gameweek 1 ---
Goalkeeper:
  - Bernd Leno
Defenders:
  - Aaron Wan-Bissaka
  - Axel Disasi
  - Joachim Andersen
  - Raphaël Varane
Midfielders:
  - Bukayo Saka
  - Harvey Barnes
  - Jarrod Bowen
  - Rodrigo Hernandez
Attackers:
  - Alexander Isak
  - Erling Haaland
Captain: Raphaël Varane (Points doubled)


--- Gameweek 2 ---
Goalkeeper:
  - Guglielmo Vicario
Defenders:
  - Lucas Digne
  - Pervis Estupiñán
  - Serge Aurier
Midfielders:
  - Bryan Mbeumo
  - Diogo Teixeira da Silva
  - Kaoru Mitoma
  - Leon Bailey
  - Solly March
Attackers:
  - Michail Antonio
  - Yoane Wissa
Captain: Bryan Mbeumo (Points doubled)


--- Gameweek 3 ---
Goalkeeper:
  - José Malheiro de Sá
Defenders:
  - Destiny Udogie
  - Joachim Andersen
  - Malo Gusto
  - Matty Cash
Midfielders:
  - Bruno Borges Fernandes
  - Jarrod Bowen
  - Moussa Diaby
  - Raheem Sterling
  - Rodrigo Hernandez
Attackers:
  - Darwin Núñez Ribeiro
Captain: Raheem Sterling (Points doubled)


--- Gameweek 4 ---
Goalkeeper:
  - Alisson

In [71]:
def print_teams(selected_players, captains, data):
    finaltotal=0
    player_name_map = {row['playerid']: row['name'] for _, row in data.iterrows()}
    player_position_map = {row['playerid']: row['position'] for _, row in data.iterrows()}
    
    for gw in range(1, 39):  
        oops = "GW_" + str(gw)
        player_points_map = {row['playerid']: row[oops] for _, row in data.iterrows()}
        print(f"--- Gameweek {gw} ---")
        
        gw_players = [p for p, gwk in selected_players.keys() if gwk == gw]
        
        if not gw_players:
            print("No players selected for this gameweek.")
            continue

        team = {"Goalkeeper": [], "Defender": [], "Midfielder": [], "Attacker": []}
        total_points = 0
        for player in gw_players:
            position = player_position_map[player]
            name = player_name_map[player]
            points = player_points_map.get(player, 0)  # Get points for this gameweek
            
            if position == 'GK':
                team["Goalkeeper"].append((name, points))
            elif position == 'DEF':
                team["Defender"].append((name, points))
            elif position == 'MID':
                team["Midfielder"].append((name, points))
            elif position == 'FWD':
                team["Attacker"].append((name, points))
            
            total_points += points  # Add player's points to total

        # Print players and their points by position
        print("Goalkeeper:")
        for gk, points in team["Goalkeeper"]:
            print(f"  - {gk} (Points: {points})")

        print("Defenders:")
        for df, points in team["Defender"]:
            print(f"  - {df} (Points: {points})")

        print("Midfielders:")
        for mf, points in team["Midfielder"]:
            print(f"  - {mf} (Points: {points})")

        print("Attackers:")
        for fw, points in team["Attacker"]:
            print(f"  - {fw} (Points: {points})")

        # Check if there's a captain and apply doubled points
        gw_captain = [p for p, gwk in captains.keys() if gwk == gw]
        if gw_captain:
            captain = gw_captain[0]
            captain_name = player_name_map[captain]
            captain_points = player_points_map.get(captain, 0)
            total_points += captain_points  # Already added once; add again for doubled points
            print(f"Captain: {captain_name} (Points: {captain_points} x2)")

        print(f"Total points for Gameweek {gw}: {total_points}")
        finaltotal+=total_points
        print("\n")
    print("Total Points over all gameweeks:", finaltotal )

# Example call to the function
print_teams(selected_players, captains, data)

--- Gameweek 1 ---
Goalkeeper:
  - Bernd Leno (Points: 12)
Defenders:
  - Cristian Romero (Points: 7)
  - Joachim Andersen (Points: 9)
  - Raphaël Varane (Points: 14)
Midfielders:
  - Bryan Mbeumo (Points: 7)
  - Jarrod Bowen (Points: 9)
  - Mohamed Salah (Points: 5)
  - Rodrigo Hernandez (Points: 13)
  - Solly March (Points: 9)
Attackers:
  - Erling Haaland (Points: 13)
  - Julián Álvarez (Points: 5)
Captain: Raphaël Varane (Points: 14 x2)
Total points for Gameweek 1: 117


--- Gameweek 2 ---
Goalkeeper:
  - Bernd Leno (Points: 2)
Defenders:
  - Cristian Romero (Points: 6)
  - Joachim Andersen (Points: 2)
  - Matty Cash (Points: 6)
Midfielders:
  - Bryan Mbeumo (Points: 16)
  - Jarrod Bowen (Points: 2)
  - Mohamed Salah (Points: 5)
  - Rodrigo Hernandez (Points: 3)
  - Solly March (Points: 15)
Attackers:
  - Erling Haaland (Points: 2)
  - Julián Álvarez (Points: 7)
Captain: Bryan Mbeumo (Points: 16 x2)
Total points for Gameweek 2: 82


--- Gameweek 3 ---
Goalkeeper:
  - Bernd Leno (Po